# Screener for buy signals

In [1]:
# imports
from scraper import stock_daily
from analyzer import Analyzer
from IPython.display import display
import datetime as dt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import indicators as ind

In [2]:
# Define buy funtion - input stock data, calculates buy signals and when last signal was received
# outputs data frame with ticker, last buy signal date (or index), distance from last buy signal

# run analysis

# outputs dataframe including all tickers, filtered by last buy signal

# maybe add % change from last signal?

In [3]:
def buysig(ticker="None"):
    # create empty series
    result = {"ticker": ticker, "buy_date": np.nan, "distance": np.nan, "change[%]": np.nan}
    result = pd.Series(data=result)
    # scraping / loading data
    try:
        stock = stock_daily(ticker, save=False)
    except:
        print(ticker + ": Exception occured during data scraping, skipped.")
        return result
    
    # calculating indicators
    vfi = ind.vfi(stock.data, period=30, coef=0.2, vcoef=1.5)
    fs = ind.stoch(stock.data, period=5, sk=2, sd=3)
    ss = ind.stoch(stock.data, period=21, sk=2, sd=5)
    # calculating VFI histogram trend
    window = 3
    vfi_hist = vfi["histogram"].rolling(window=window).apply(lambda x: np.polyfit(np.arange(window), x, 1)[0], raw=True).values
    vfi_hist = vfi_hist > 0
    # VFI trend
    vfi_trend = vfi["vfi"] > vfi["vfi_smooth"]
    vfi_conf = np.logical_or(vfi_hist, vfi_trend)
    # calculating fast stochastic trend
    window = 4
    fs_conf = fs["k"].rolling(window=window).apply(lambda x: np.polyfit(np.arange(window), x, 1)[0], raw=True).values
    fs_conf = fs_conf > 0
    # buy signals
    conditions = np.logical_and((ss["k"] > ss["d"]).to_numpy(), (ss["d"] >= 0).to_numpy())
    bss = np.concatenate((np.array([0]), (conditions[:-1] < conditions[1:]))).astype("int")
    # finalize buy signals
    bsf = np.logical_and(np.logical_and(fs_conf, vfi_conf), bss)
    if bsf.sum() == 0:
        print(ticker + ": No buying signals generated, skipped.")
        return result
    
    # date of last buy signal and distance from today
    try:
        lastindex = np.squeeze(np.where(bsf == True))[-1]
        result["buy_date"] = stock.data.loc[lastindex,"Date"]
        result["distance"] = stock.data.index[-1] - lastindex
    except:
        print("Something broke")
        return result

    return result

In [4]:
iwm = pd.read_excel("spy.xlsx")
tickers = iwm["ticker"].to_list()

# define empty dataframe
data = pd.DataFrame(columns=["ticker", "buy_date", "distance", "change[%]"])
# run analysis
for i in tickers:
    res = buysig(i).to_frame().T
    data = data.append(res, ignore_index=True)

- BRK.B: No data found, symbol may be delisted
BRK.B: No buying signals generated, skipped.
BBWI: No buying signals generated, skipped.
- BF.B: No data found for this date range, symbol may be delisted
BF.B: No buying signals generated, skipped.
Something broke


In [5]:
display(data[data["distance"] <= 1].reset_index(drop=True))

,ticker,buy_date,distance,change[%]
0,PYPL,2021-09-07,0,NaN
1,WMT,2021-09-03,1,NaN
2,ADI,2021-09-07,0,NaN
3,MPC,2021-09-07,0,NaN
4,IT,2021-09-02,1,NaN
5,KSU,2021-09-02,1,NaN
6,RSG,2021-09-02,1,NaN
7,DAL,2021-09-03,0,NaN
8,FTV,2021-09-03,0,NaN
9,OKE,2021-09-03,0,NaN
